<a href="https://colab.research.google.com/github/aelydens/aie-resources/blob/main/AL_RAG_NVIDIA_10k_Filings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NVIDIA 10k Filings RAG with Langchain

Deliverables:

Build 🏗️
* Data: NVIDIA 10-k Filings
* Model: OpenAI text-3-embedding small, GPT-3.5-turbo
* Tooling: LangChain or LlamaIndex (you choose)
* Vector Store: FAISS
* Additional Component: Add one of the following: 1) visibility with WandB OR 2) evaluation with RAGAS
Ship 🚢

Evaluate your answers to the following questions
* "Who is the E-VP, Operations - and how old are they?"
* "What is the gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023?"
* Record <10 min loom video walkthrough

In [61]:
!pip install -qU langchain langchain-core langchain-community langchain-openai pymupdf faiss_cpu

In [62]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


Initialize our embedding model to be `text-embedding-3-small`.

In [63]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Load and split our documents.

In [64]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyMuPDFLoader("https://d18rn0p25nwr6d.cloudfront.net/CIK-0001045810/1cbe8fe7-e08a-46e3-8dcc-b429fc06c1a4.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

Set up our FAISS-powered vector store, and create a retriever.

In [65]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

retriever = vector_store.as_retriever()

Create our prompt using `ChatPromptTemplate`.

In [66]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

Initialize our OpenAI model with a temperature of 0.

In [67]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

Create our LCEL chain.

In [68]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | openai_chat_model, "context": itemgetter("context")}
)


Question time!

In [69]:
question = "Who is the E-VP, Operations - and how old are they?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Debora Shoquist is the Executive Vice President of Operations, and she is 69 years old.


In [ ]:
question = "What is the gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023 in billions?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

$3.539 billion


# RAGAS Evaluation

Split the documents to create our `eval_documents` set.

In [70]:
eval_documents = documents

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2500,
    chunk_overlap = 400
)

eval_documents = text_splitter.split_documents(eval_documents)

In [71]:
!pip install -qU ragas

Generate our test question set.

In [72]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator = TestsetGenerator.with_openai()

generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-4")

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

testset = generator.generate_with_langchain_docs(documents, 10, distributions)
testset.to_pandas()

<ipython-input-72-8de02e98e699>:5: DeprecationWarning: The function with_openai was deprecated in 0.1.4, and will be removed in the 0.2.0 release. Use from_langchain instead.
  generator = TestsetGenerator.with_openai()


embedding nodes:   0%|          | 0/246 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What products are included in the GeForce RTX ...,[Table of Contents\nAt the foundation of the N...,The GeForce RTX line for the gaming market inc...,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
1,What is the process of foreign currency remeas...,[Table of Contents\nNVIDIA Corporation and Sub...,The process of foreign currency remeasurement ...,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
2,What impact do USG export controls have on bus...,[Table of Contents\nGlobal Trade\nDuring the t...,The USG export controls have harmed the compan...,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
3,How is goodwill subject to an impairment test?,[Table of Contents\nNVIDIA Corporation and Sub...,Goodwill is subject to our annual impairment t...,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
4,What is the purpose and definition of internal...,[Table of Contents\nReport of Independent Regi...,The purpose of internal control over financial...,simple,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
5,What risks and challenges come with using new ...,[Table of Contents\nregulator or other industr...,The use of new technologies like AI responsibl...,multi_context,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
6,What are the risks and benefits of business in...,[Table of Contents\nWe are monitoring the impa...,The risks and benefits of business investments...,multi_context,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
7,How does NVIDIA recognize revenue from license...,[Table of Contents\nNVIDIA Corporation and Sub...,NVIDIA recognizes revenue from license and dev...,multi_context,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
8,What is the purpose of the Compensation Recove...,[ \nCOMPENSATION RECOVERY POLICY\nLast ...,The purpose of the Compensation Recovery Polic...,multi_context,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True
9,What are GPUs in the NVIDIA accelerated comput...,[Table of Contents\nAt the foundation of the N...,GPUs in the NVIDIA accelerated computing platf...,reasoning,[{'source': 'https://d18rn0p25nwr6d.cloudfront...,True


In [74]:
test_df = testset.to_pandas()


Format our resulting test questions and answers into a Hugging Face dataset so we can use it with the Ragas library.

In [76]:
from datasets import Dataset

test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

/usr/lib/python3.10/json/decoder.py:353: RuntimeWarning: coroutine 'process_batches' was never awaited
  obj, end = self.scan_once(s, idx)


Evaluate our responses using metrics from RAGAS.

In [77]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

results = evaluate(response_dataset, metrics)

results

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

{'faithfulness': 0.8800, 'answer_relevancy': 0.9402, 'context_recall': 0.9500, 'context_precision': 0.9139, 'answer_correctness': 0.6847}

Finally, we can view our results in a dataframe.

In [78]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What products are included in the GeForce RTX ...,GeForce RTX GPUs for gaming desktop and laptop...,[Table of Contents\nAt the foundation of the N...,The GeForce RTX line for the gaming market inc...,0.5,0.910411,1.0,1.000000,0.666817
1,What is the process of foreign currency remeas...,Foreign currency remeasurement involves conver...,[Table of Contents\nNVIDIA Corporation and Sub...,The process of foreign currency remeasurement ...,1.0,0.973649,1.0,0.638889,0.998882
2,What impact do USG export controls have on bus...,The USG export controls negatively impact busi...,"[Table of Contents\nproducts, and it is likely...",The USG export controls have harmed the compan...,1.0,0.841474,1.0,1.000000,0.488319
3,How is goodwill subject to an impairment test?,Goodwill is subject to an impairment test eith...,[Table of Contents\nNVIDIA Corporation and Sub...,Goodwill is subject to our annual impairment t...,1.0,0.915788,1.0,1.000000,0.615695
4,What is the purpose and definition of internal...,The purpose of internal control over financial...,[Table of Contents\nReport of Independent Regi...,The purpose of internal control over financial...,0.8,0.975548,1.0,1.000000,0.435312
5,What risks and challenges come with using new ...,The risks and challenges of using new technolo...,[Table of Contents\nregulator or other industr...,The use of new technologies like AI responsibl...,1.0,0.971845,1.0,1.000000,0.619200
6,What are the risks and benefits of business in...,The risks of business investments and acquisit...,[Table of Contents\n•\nInternational sales and...,The risks and benefits of business investments...,1.0,0.929081,1.0,0.500000,0.705634
7,How does NVIDIA recognize revenue from license...,NVIDIA recognizes revenue from license and dev...,[Table of Contents\nRevenue Recognition\nReven...,NVIDIA recognizes revenue from license and dev...,1.0,0.969689,1.0,1.000000,0.618830
8,What is the purpose of the Compensation Recove...,The purpose of the Compensation Recovery Polic...,[ \nCOMPENSATION RECOVERY POLICY\nlaw o...,The purpose of the Compensation Recovery Polic...,1.0,0.963619,1.0,1.000000,0.849096
9,What are GPUs in the NVIDIA accelerated comput...,GPUs in the NVIDIA accelerated computing platf...,[Table of Contents\nResearchers and developers...,GPUs in the NVIDIA accelerated computing platf...,0.5,0.950551,0.5,1.000000,0.849062
